<a href="https://colab.research.google.com/github/jmyth742/EEG-BCI/blob/master/WorkingLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pydrive

In [0]:
import numpy as np
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
import time



##### START OF ADDITION OF MY CODE

import numpy as np
import os.path
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import io
from googleapiclient.http import MediaIoBaseDownload

from scipy.io import loadmat
from pathlib import Path
import matplotlib.pyplot as plt
import scipy.io as sio
import sys

## PyTorch 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch import Tensor
import torch.optim as optim
import torchvision

import math #for calculus

In [0]:

## END OF IMPORTS
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#file_id = '1FrXb6rTyqpE5SmNtP8HhygDmFf9Lw896'
file_id = '1al08tF3j-Z2-fowjPPNz1SrFgLVqywXG' ##1.mat
#https://drive.google.com/open?id=1TqewoCjjRXZpEIxL_23ZQ8MP4L0RofZx
file_id = '1TqewoCjjRXZpEIxL_23ZQ8MP4L0RofZx'
#https://drive.google.com/open?id=1al08tF3j-Z2-fowjPPNz1SrFgLVqywXG
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('./train3.mat')
#mat = loadmat('train3.mat')
mat = sio.loadmat('train3.mat', squeeze_me=True, struct_as_record=False)
data=[]

o=mat['o']

data=mat['o'].data
data_set=mat['o'].data
#print(data[200][21])
labels=mat['o'].marker

newlabels = np.zeros([data.shape[0],4])

pos=np.zeros((22,667000), dtype=int)
found = np.zeros([1,])
#clean the shit out of it
while np.amax(labels) > 3:
    ind = np.argmax(labels)
    found = np.append(found,np.array([ind]), axis=0)
    labels[ind] = 0
    data[ind,:] = np.zeros([22,])
    
   
#print(np.amax(labels))
#print(found.shape)
#print(found)

inside = 0
for y in labels:
  if inside % 10000 == 0:
    #print(inside)
    newlabels[inside,y] = 1
    inside += 1

labeled_data = (data,labels)

In [0]:
data = np.hstack((data,labels.reshape((labels.shape[0],1))))
data = data.astype('float32')

In [0]:
class eeg_LSTM(nn.Module):
    def __init__(self):
        super(eeg_LSTM, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.lstm = nn.LSTM(22, 4, 2)
        #torch.nn.init.xavier_normal_(self.lstm.weight)
        
    def forward(self, x):
        if hidden==None:
          hidden = self.init_hidden(input.shape[1])
          
        if cell==None:
          cell = self.init_hidden(input.shape[1])
          
        output, (_,_)= self.i2h(input, (hidden,cell))
        r_out, _ = self.lstm(input, (hidden,cell))
        output = F.softmax(r_out) 
        return output
      
      
    def init_hidden(self,shape=1):
        return torch.zeros(1, shape, self.hidden_size)
      
    def init_cell(self,shape=1):
        return torch.zeros(1, shape, self.hidden_size)
      


In [0]:
def get_optimizer(net, lr):
  optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
  #oprimizer = torch.optim.SGD(net.parameters(), lr=lr, weight_decay=wd, momentum=momentum)
  return optimizer

def get_cost_function():
  cost_function = torch.nn.CrossEntropyLoss()
  return cost_function

In [0]:
def get_data(batch_size,test_batch_size=256,step=1):
  full_data = data
  #print(full_data.shape)
  # Create train and validation splits
  num_samples = len(full_data)
  training_samples = int(num_samples)
  #validation_samples = num_samples - training_samples  
  

  training=data[:training_samples]
  #validation=data[training_samples:]

  # Initialize dataloaders
  train_loader = torch.utils.data.DataLoader(training, batch_size, shuffle=False)
  #val_loader = torch.utils.data.DataLoader(validation, test_batch_size, shuffle=False)
  #test_loader = torch.utils.data.DataLoader(test_data, test_batch_size, shuffle=False)
  
  return train_loader#, val_loader

In [0]:
def test(time_seq, batch_size, net, data_loader, cost_function, device='cuda:0'):
  samples = 0.
  cumulative_loss = 0.
  cumulative_accuracy = 0.
  net.eval() # Strictly needed if network contains layers which has different behaviours between train and test
  with torch.no_grad():
    for batch_idx, (inputs) in enumerate(data_loader):
      # Load data into GPU
      xtrain = inputs[:,0:22]
      labels = inputs[:,22]
      train = Variable(torch.FloatTensor(xtrain.reshape(time_seq,batch_size,22))).to(device)#.reshape(batch size, seq len, channels)
      labels = inputs[:,22]
      
      #print("shape of labels", labels.shape)
      #print("Contents of labels", labels)
      
      output_seq = net(train)
      last_output = output_seq[-1]
      
      targets = Variable(labels.long()).to(device)
      loss = cost_function(last_output, targets)
      
      # Better print something
      samples+=inputs.shape[0]
      cumulative_loss += loss.item() # Note: the .item() is needed to extract scalars from tensors
      _, predicted = last_output.max(1)
      cumulative_accuracy += predicted.eq(targets).sum().item()

  return cumulative_loss/samples, cumulative_accuracy/samples*100


def train(time_seq, batch_size,net,data_loader,optimizer,cost_function, device='cuda:0'):
  samples = 0.
  cumulative_loss = 0.
  cumulative_accuracy = 0.
  
  net.train() # Strictly needed if network contains layers which has different behaviours between train and test
  #for i in range((step//batch_size)-1):
  for batch_idx, (inputs) in enumerate(data_loader):
     # Load data into GPU
      xtrain = inputs[:,0:22]
      labels = inputs[:,22]
      train = Variable(torch.FloatTensor(xtrain.reshape(time_seq,batch_size,22))).to(device)#.reshape(batch size, seq len, channels)
      
      #input_seq = Variable(torch.randn(time_steps, batch_size, in_size))
      output_seq = net(train)
      last_output = output_seq[-1]
      
      #targets = Variable(torch.LongTensor(batch_size).random_(0, 3)).to(device)
      targets = Variable(labels.long()).to(device)
      loss = cost_function(last_output, targets)
      
      # Better print something
      samples+=inputs.shape[0]
      cumulative_loss += loss.item() # Note: the .item() is needed to extract scalars from tensors
      _, predicted = last_output.max(1)
      cumulative_accuracy += predicted.eq(targets).sum().item()

  return cumulative_loss/samples, cumulative_accuracy/samples*100


In [0]:
'''
Input arguments
  batch_size: Size of a mini-batch
  device: GPU where you want to train your network
  weight_decay: Weight decay co-efficient for regularization of weights
  momentum: Momentum for SGD optimizer
  epochs: Number of epochs for training the network
'''

def main(batch_size=3335, 
         device='cuda:0', 
         learning_rate=0.0000000001, 
         time_seq=1,
         epochs=500):
  
  #train_loader, val_loader = get_data(batch_size,batch_size)
  train_loader = get_data(batch_size,batch_size)
  net = eeg_LSTM().to(device)
  
  optimizer = get_optimizer(net, learning_rate)
  
  cost_function = get_cost_function()

  print('Before training:')
  train_loss, train_accuracy = test(time_seq, batch_size, net, train_loader, cost_function)
  #val_loss, val_accuracy = test(net, val_loader, cost_function)
  #test_loss, test_accuracy = test(net, test_loader, cost_function)

  print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
  #print('\t Validation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
  #print('\t Test loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
  print('-----------------------------------------------------')

  for e in range(epochs):
    #train_loss, train_accuracy = train(time_se1, batch_size,net, train_loader, optimizer, cost_function)
    #val_loss, val_accuracy = test(net, val_loader, cost_function)
    print('Epoch: {:d}'.format(e+1))
    print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
    #print('\t Validation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
    print('-----------------------------------------------------')

  print('After training:')
  #train_loss, train_accuracy = test(net, train_loader, cost_function)
  #val_loss, val_accuracy = test(net, val_loader, cost_function)
  #test_loss, test_accuracy = test(net, test_loader, cost_function)

  print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
  #print('\t Validation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
  #print('\t Test loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
  print('-----------------------------------------------------')


In [0]:
main()